In [491]:
import openml
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt

import torch
import sklearn

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms

In [471]:
        openml_ids = {
            'iris': 61,  # 150 x 5 - 3 (n_samples x n_feat - n_classes)
            'pendigits': 32,  # 10992 x 17 - 10
            'glass': 41,  # 214 x 10 - 7 (really 6)
            'segment': 36,  # 2310 x 19 - 7
            'vehicle': 54,  # 846 x 19 - 4
            'vowel': 307,  # 990 x 13 - 11 <- This has cat feat to deal with
            'wine': 187,  # 178 x 14 - 3
            'abalone': 1557,  # 4177 x 9 - 3 <- Firts cat feat
            'balance-scale': 11,  # 625 x 5 - 3
            'car': 21,  # 1728 x 7 - 4 <- All categoric
            'ecoli': 39,  # 336 x 8 - 8
            'satimage': 182,  # 6435 x 37 - 6
            'collins': 478,  # 500 x 5 - 6 <- The last feat is cat (but is number as str)
            'cardiotocography': 1466,  # 2126 x 35 - 10
            'JapaneseVowels': 375,  # 9961 x 14 - 9
            'autoUniv-au6-1000': 1555,  # 1000 x 300 - 4 <- This has cat feat to deal with
            'autoUniv-au6-750': 1549,  # 750 x 300 - 4  <- This has cat feat to deal with
            'analcatdata_dmft': 469,  # 797 x 4 - 6  <- This has cat feat to deal with
            'autoUniv-au7-1100': 1552,  # 1100 x 12 - 5 <- This has cat feat to deal with
            'GesturePhaseSegmentationProcessed': 4538,  # 9873 x 32 - 5
            'autoUniv-au7-500': 1554,  # 500 x 300 - 4  <- This has cat feat to deal with
            'mfeat-zernike': 22,  # 2000 x 48 - 10
            'zoo': 62,  # 101 x 16 - 7 <- This has dichotomus feat to deal with
            'page-blocks': 30,  # 5473 x 10 - 5
            'yeast': 181,  # 1484 x 8 - 10
            'flags': 285,  # 194 x 29 - 8  <- This has cat feat to deal with
            'visualizing_livestock': 685,  # 280 x 8 - 3 <- This has cat feat to deal with
            'diggle_table_a2': 694,  # 310 x 8 - 10
            'prnn_fglass': 952,  # 214 x 9 - 6
            'confidence': 468,  # 72 x 3 - 6
            'fl2000': 477,  # 67 x 15 - 5
            'blood-transfusion': 1464,  # 748 x 4 - 2
            'banknote-authentication': 1462,  # 1372 x 4 - 2
            'cifar10': 40927, # 60000 x (32x32x3=3072) - 10
            'breast-tissue': 15,  # 699 x 9 - 2
            'cholesterol': 141,  # 10^6 x 18 - 4  <- This has cat feat to deal with
            'liver-disorders': 145,  # 10^6 x 12 - 11  <- This has cat feat to deal with
            'pasture': 294,  # 6435 x 36 - 6
            'eucalyptus': 180,  # 110393 x 54 - 7 <- This has cat feat to deal with
            'dermatology': 35,  # 366 x 34 - 6 #X = X.astype(np.float64).values should be used
            'optdigits': 28,  # 5620 x 64 - 10
            'cmc': 23,  # 1473 x 9 - 3
            }

In [495]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.__dict__['MNIST'](
    root='Datasets/raw_datasets', 
    train=True, 
    transform=transform, 
    download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting Datasets/raw_datasets\MNIST\raw\train-images-idx3-ubyte.gz to Datasets/raw_datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting Datasets/raw_datasets\MNIST\raw\train-labels-idx1-ubyte.gz to Datasets/raw_datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting Datasets/raw_datasets\MNIST\raw\t10k-images-idx3-ubyte.gz to Datasets/raw_datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting Datasets/raw_datasets\MNIST\raw\t10k-labels-idx1-ubyte.gz to Datasets/raw_datasets\MNIST\raw



In [496]:
test_dataset = datasets.__dict__['MNIST'](
    root='Datasets/raw_datasets', 
    train=False, 
    transform=transform, 
    download=True)

In [498]:
train_dataset.data.to(torch.float32).shape

torch.Size([60000, 28, 28])

In [499]:
torch.max(torch.unique(train_dataset.targets))+1

tensor(10)

In [500]:
len(torch.unique(train_dataset.targets))

51592

In [501]:
torch.unique(train_dataset.targets)

tensor([0, 7, 0,  ..., 5, 6, 8])

In [505]:
torch.unique(train_dataset.targets, sorted=True, return_inverse=True, return_counts=False, dim=None)

(tensor([0, 6, 5,  ..., 5, 6, 8]),
 tensor([   53,     1,     2,  ..., 51219, 51220, 51221]))

In [507]:
len(np.unique(train_dataset.targets))

10

In [508]:
train_num_samples = train_dataset.data.shape[0]
train_dataset.data.to(torch.float32).view((train_num_samples,-1))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [510]:
le = sklearn.preprocessing.LabelEncoder()

train_dataset.targets

tensor([5, 0, 4,  ..., 5, 6, 8])